In [1]:
author="Jumabek Alikhanov"
date = 'Nov 19,2019'

In [6]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# load data
dataroot = 'data/'
SEED=2

In [14]:
%autoreload 2
from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

data/*.csv
['data/ISCX_Botnet-Testing.pcap_Flow_labelled.csv', 'data/ISCX_Botnet-Training.pcap_Flow_labelled.csv']
there are 677598 flow records with 84 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries


ValueError: could not convert string to float: '10.0.2.15-208.73.211.152-1044-80-6'

In [5]:
from preprocessing import balance_data, normalize
X = normalize(X)


In [6]:
# Random Foresty with 10 DecisionTrees
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

K=5
results = {}
skf = StratifiedKFold(n_splits=K,random_state=SEED)

for fold_index, (dev_index,test_index) in enumerate(skf.split(X,y)): 
    print('---------------------------------------------')
    print('Fold #{}'.format(fold_index))    
    X_dev = X[dev_index]
    y_dev = y[dev_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
    for train_index, val_index in sss.split(X_dev, y_dev): # runs only once
        X_train = X_dev[train_index]
        y_train = y_dev[train_index]
        X_val = X_dev[val_index]
        y_val = y_dev[val_index]
        break
    
    #X_train,y_train = balance_data(X_train,y_train,seed=SEED)
    clf = RandomForestClassifier(n_estimators=10, random_state=SEED)
    clf.fit(X_train,y_train)
    
    pred = clf.predict(X_val)
    imbalanced_val_acc = metrics.accuracy_score(y_val,pred)*100        
    val_acc = metrics.balanced_accuracy_score(y_val,pred)*100

    pred = clf.predict(X_test)
    test_acc = metrics.balanced_accuracy_score(y_test,pred)*100
    print('val acc:',imbalanced_val_acc)
    print('balanced val acc: ',val_acc)
    print('balanced test acc: ',test_acc)
    
    results[fold_index]= (imbalanced_val_acc, val_acc, test_acc)
    


---------------------------------------------
Fold #0
val acc: 99.84169319832729
balanced val acc:  80.74039620152236
balanced test acc:  75.4521833362492
---------------------------------------------
Fold #1
val acc: 99.83506984692627
balanced val acc:  84.20717638304129
balanced test acc:  79.73577560233268
---------------------------------------------
Fold #2
val acc: 99.91278813035414
balanced val acc:  85.23290907807385
balanced test acc:  81.71346414237206
---------------------------------------------
Fold #3
val acc: 99.83992758102977
balanced val acc:  84.58250011448195
balanced test acc:  79.65610580982644
---------------------------------------------
Fold #4
val acc: 99.85781153404575
balanced val acc:  82.99806506021838
balanced test acc:  74.00753893558011


In [7]:
sum_imbalanced_val_acc = 0
sum_val_acc = 0
sum_test_acc = 0
for fold_index,res in results.items():
    (acc,balanced_acc,test_acc) = res
    sum_imbalanced_val_acc+=acc
    sum_val_acc +=balanced_acc
    sum_test_acc+= test_acc
print("{}-fold evaluation:".format(K))
print('imbalanced val acc:',sum_imbalanced_val_acc/K)
print('val acc: ',sum_val_acc/K)
print('test acc: ',sum_test_acc/K)
print('{0:.2f} {1:.2f} {2:.2f}'.format(sum_imbalanced_val_acc/K,sum_val_acc/K,sum_test_acc/K))

5-fold evaluation:
imbalanced val acc: 99.85745805813664
val acc:  83.55220936746755
test acc:  78.1130135652721
99.86 83.55 78.11
